虽然 CSV 文件读写很方便，但通常我们更希望写到数据库中。你可以选择传统的关系型数据库（MySQL，Postgresql，Sqlite 等），也可以选择新型的文档数据库（Mongodb 等）。要用 Python 连接数据库，当然要有数据库驱动，MySQL 的驱动是 MySQLdb；Postgresql 的驱动是 Psycopg2；Sqlite 的驱动是 pysqlite（等同于 sqlite3）；Mongodb 的驱动是 PyMongo。每个数据库都有专门的驱动，这样就会导致代码的可维护性很差，所以，通常我们都会选择 ORM（Object Relational Mapper）模型 SQLAlchemy 来保存数据。SQLAlchemy 是用 Python 实现的针对关系数据库的 ORM 模型，可以很容易地在不同数据库切换（Python 中有很多 ORM 实现，感兴趣的同学可以查阅相关资料）。SQLite 是一个轻量级的数据库，占用资源很少，不需要繁琐的配置，数据库文件只有一个，很容易备份、转移。通常情况下 SQLite 都是一个比较好的选择。更神奇的是，SQLite 还支持内存数据库，非常有利于教学，所以，我们在本教程中选择使用 SQLite 数据库。

对象关系映射（Object-Relational Mapping）提供了概念性的、易于理解的模型化数据的方法。ORM方法论基于三个核心原则： 简单：以最基本的形式建模数据。 传达性：数据库结构被任何人都能理解的语言文档化。 精确性：基于数据模型创建正确标准化的结构。 典型地，建模者通过收集来自那些熟悉应用程序但不熟练的数据建模者的人的信息开发信息模型。建模者必须能够用非技术企业专家可以理解的术语在概念层次上与数据结构进行通讯。建模者也必须能以简单的单元分析信息，对样本数据进行处理。ORM专门被设计为改进这种联系。简单的说：ORM相当于中继数据。


SQLAlchemy是一个基于Python实现的ORM框架。该框架建立在 DB API之上，使用关系对象映射进行数据库操作，简言之便是：将类和对象转换成SQL，然后使用数据API执行SQL并获取执行结果。

SQLAlchemy本身无法操作数据库，其必须以来pymsql等第三方插件。

In [1]:
from sqlalchemy import create_engine

#engine = create_engine(
#        "mysql+pymysql://",
#        encoding="UTF-8",
#        max_overflow=0,  # 超过连接池大小外最多创建的连接
#        pool_size=5,  # 连接池大小
#        pool_timeout=30,  # 池中没有线程最多等待的时间，否则报错
#        pool_recycle=-1  # 多久之后对线程池中的线程进行一次连接的回收（重置）
#    )

#print (engine.driver)
#print (engine.name, engine.dialect) # engine.name 是 engine.dialect 的名称

In [2]:
engine = create_engine('sqlite+pysqlite://', encoding="UTF-8", echo=False)
print (engine.driver)
print (engine.name, engine.dialect) # engine.name 是 engine.dialect 的名称
print (engine.table_names()) # 列出数据库中所有的表

pysqlite
sqlite <sqlalchemy.dialects.sqlite.pysqlite.SQLiteDialect_pysqlite object at 0x7f0732424690>
[]


上面的代码创建了一个 Engine的实例和一个使用 UTF-8编码的内存数据库，Engine 是 SQLAlchemy 应用的起点，管理着数据库的连接和 SQL 的执行。如果你需要创建一个磁盘数据库，只需要指定一个路径就可以了，如：create_engine('sqlite:////absolute/path/to/foo.db')。echo参数为 True 时，会显示每条执行的SQL语句，通常在调试阶段打开。

可以看到，sqlalchemy 位于数据库驱动，内部仍然会调用驱动的 API 来实现真正的数据库操作，如上面的驱动默认采用的就是pysqlite。我们知道每个数据库所使用的 SQL（称为方言）都略有不同，但我们不可能记住这些细微的差别，SQLAlchemy 已经为我们隐藏了这些细节，它会根据数据库的不同自动选择不同的方言，如上面的 SQLiteDialect_pysqlite 就是专门针对 SQLite 的方言。如果你愿意，你可以用上面得到的 engine 直接进行数据库操作：

In [4]:
engine.execute('create table dac2020(id int, name varchar(20));')
print (engine.table_names())
engine.execute('insert into dac2020 values(1, "Tom");')
engine.execute('insert into dac2020 values(2, "Lucy");')
res = engine.execute('select * from dac2020;')
for i in res:
    print (i)
res.close()
engine.execute('drop table dac2020;')
print (engine.table_names())

['dac2020']
(1, 'Tom')
(2, 'Lucy')
[]


上面的代码中，我们首先创建了一个表 test，然后插入了两条数据并查询出这两条数据，最后删除表 test。我们还可以直接用 engine 实现事务，参考 Engine.begin，这里不再赘述。可以看到，上面的代码中可以实现我们想要的功能，但我们仍然使用了 SQL 查询，这与直接使用数据库驱动没有什么区别！总之，你可以利用 engine 直接进行数据库操作，但完全没有体现出 ORM 的威力。

### 创建映射模型

使用 ORM 时，首先要定义模型（其实就是常规的 Python 类），这些模型是数据库表的映射。程序启动时，SQLAlchemy 会根据模型自动创建需要的数据库表（自动跳过已经跳过的数据库表）。创建模型的方法如下：

In [6]:
from sqlalchemy import Column
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.types import Integer, String

# 创建一个所有模型的基类
BaseModel = declarative_base() 
# 定义自己的模型
class Stu(BaseModel):
    __tablename__ = 'dac2020'
    id = Column(Integer, primary_key=True) # 默认 autoincrement=True
    name =  Column(String(50))
    
    def __init__(self, _id, name):
        self.id = _id
        self.name = name

    def __str__(self):
        return str(self.id) + ': ' + str(self.name)

    def __repr__(self):
        
        return '<Stu(%d, %s)>' % (self.id, self.name)

# 创建所有模型对应的数据库表
BaseModel.metadata.create_all(engine)
print (engine.table_names())
print (engine.execute('pragma table_info(' + Stu.__tablename__ +')').fetchall())

['dac2020']
[(0, 'id', 'INTEGER', 1, None, 1), (1, 'name', 'VARCHAR(50)', 0, None, 0)]


上面的代码中，首先用 declarative_base 方法创建了一个 BaseModel 类，这是所有模型的父类，它的子类可以自动与一个表关联。接着以 BaseModel 为基础创建了一个 User 模型，其中的 __tablename__ 属性就是 User 对应的数据库表的名称。User 中还有 id 和 name 两个属性，都是 Column 类型。Column 表示这个属性是一个数据库字段，其中定义了这个字段的类型，是否是主键，是否自增等属性。如我们定义的User 模型中 id 整型，并且是主键，name 是长度不超过 50 的字符串，相当于 VARCHAR(50)。最后我们调用 create_all 方法会找到BaseModel 的所有子类，并在数据库中创建所有的数据库表。可以看到，这个方法结束之后，数据库中确实创建了 user 表。需要说明的是，pragma是SQLite 的特有语法（这就是前面提到过的方言）。

除了Integer，String，SQLAlchemy 还支持很多其它的类型，如 Boolean，TEXT，Date等。

### 创建会话
定义了模型之后就准备好和数据库交互了。不过在进行实际的操作之前，我们还要做一些准备工作，即创建一个会话：

In [7]:
from sqlalchemy.orm import sessionmaker
DB_Session = sessionmaker(bind=engine)
session = DB_Session()

session 主要有以下几个作用：

    管理数据库连接。
    管理数据缓存和持久化操作，保证数据在内存和磁盘中的一致性。
    提供模型到数据库表的操作转换。

sessionmaker 是一个工厂方法，用于创建一个和指定的 engine 绑定的 Session 类。通常一个应用拥有一个会话就够了。通过 session 对象提供的 api 我们就可以通过操作我们定义的模型来操作数据库。

### 插入数据

下面是向数据库中插入数据的方法： 

In [9]:
session.execute('delete from dac2020;') # 清空数据表 user
stu1 = Stu(1, 'Tom')
stu2 = Stu(2, 'Lucy')
stu3 = Stu(3, 'Lily')
session.add(stu1) # 插入一条数据
session.add_all([stu2, stu3]) # 批量插入

session.flush() # 将改动写到磁盘
print (session.execute('select * from dac2020;').fetchall())

[(1, 'Tom'), (2, 'Lucy'), (3, 'Lily')]


可以看到，我们成功向数据库表 user 中插入了三条数据。

### 查询数据 
到目前为止，我们对数据库的查询，采用的都是直接执行 SQL 语句的方法，更方便的是下面的方法：

In [12]:
# 选择 user 表中的所有记录。
print (session.query(Stu).all())
# 指定排序字段。
print (session.query(Stu).order_by(Stu.name).all())
# get 主键为 1 的记录
print (session.query(Stu).get(1))
# 意义同上。条件过滤，相当于 where 字句。
print (session.query(Stu).filter_by(id=1).first())
# 意义同上。但 one 的含义是：数据库中有且只能有一条满足条件的记录，否则就会抛出异常
print (session.query(Stu).filter_by(id=1).one() is stu1)
# 用 limit 限制返回记录数
print (session.query(Stu).filter(Stu.id>1).limit(2).all() )
# 组合过滤条件，相当于将多个 Where 字句 用 and 起来
print (session.query(Stu).filter(Stu.id>1).filter(Stu.name.in_(['Tom', 'Lucy'])).all())
# OR 子句
from sqlalchemy import or_
print (session.query(Stu).filter(or_(Stu.name.like('To%'), Stu.id.between(2,3))).count())

[<Stu(1, Tom)>, <Stu(2, Lucy)>, <Stu(3, Lily)>]
[<Stu(3, Lily)>, <Stu(2, Lucy)>, <Stu(1, Tom)>]
1: Tom
1: Tom
True
[<Stu(2, Lucy)>, <Stu(3, Lily)>]
[<Stu(2, Lucy)>]
3


上面的查询充分利用了 SQLAlchemy 的 ORM 特性。 让我们用近 Python 语法就可以执行 SQL 查询。首先我们用 session.query(User)，得到一个 Query 对象，然后给这个 query 添加一些查询条件（filter，limit，order_by 等），最后获得查询的结果（all，first，one，count 等）。Query 对象中包含很多查询操作，除了上面我们涉及的，还支持 group_by，union，join 等。SQLAlchemy 还支持很多过滤条件，除了上面涉及到的，还有IS NULL，不等，AND 等。值得注意的是程序的第8行，我们获得了 id=1的用户，可以看到，我们查询出的对象是和前面插入的对象 user1 是同一个，这是因为 SQLAlchemy 会给每一条记录缓存一个实例，以提高性能。

怎么样？这种查询方法是不是比直接用 SQL 方便多了。

### 更新数据

有时我们还需要修改数据，修改数据非常简单，直接在数据对象上修改，然后调用 save 即可。 

In [14]:
stu = session.query(Stu).filter_by(id=1).one()
# 把 user 的 name 改成 Thomas
stu.name = 'Thomas'
session.flush()
print (session.query(Stu).filter_by(id=1).one())
# 撤销修改
stu.name = 'Tom'
session.flush()
print (session.query(Stu).filter_by(id=1).one())
session.commit() # 提交一次，保存前面所有的修改，因为后面要讲到回滚，防止数据丢失。

1: Thomas
1: Tom


上面代码中我们首先选择出id=1的记录user，然后直接修改 user 的 name 属性，最后调用 session.flush() 就可以了。最后我们又撤销了修改并提交当前的所有修改。

### 删除数据

利用 SQLAlchemy 删除记录也很简单： 

In [17]:
stu = session.query(Stu).filter_by(id=1).one()
session.delete(stu)
print (session.deleted) # deleted 是一个集合，记录了通过这个 session 删除的对象。
session.flush()
print (session.deleted) # flush 之后 deleted 集合就会清空
print (session.query(Stu).filter_by(id=1).first())
session.commit()

IdentitySet([<Stu(1, Tom)>])
IdentitySet([])
None


删除一个记录果然很简单。但一失足成千古恨，这是一条很重要的数据，结果我却把它删除了，呜呜呜~。别着急，有办法恢复，因为我们还处在一个事务当中没有提交呢！只需要执行 rollback 就可以了：

In [18]:
session.rollback()
print (session.query(Stu).filter_by(id=1).first())

None


哇，事务真是个好东西！接下来我们就学习一下 SQLAlchemy 中的事务处理。

### 事务处理

也许你没有注意到，在前面的讲解中我们已经开始使用事务了，因为一个 session 开始的时候会自动开始一个事务。前面我们将数据写入数据库使用的都是 session.flush 方法，但 flush 只是将内存中的数据写到磁盘上，并不会提交修改，所以在 "删除数据" 那一小节中，我们删除数据后虽然执行了 flush，但我们仍然可以通过后续的 rollback 恢复数据。提交一个事务的方法是执行 session.commit 方法，commit 不但会自动执行 flush，还会提交当前所有修改，防止数据丢失。这正是我们在 "更新数据" 那一小节的最后一行执行 session.commit 的目的。commit 之后，session 会自动开始一个新的事务。所以，事务处理就是为了在出错的时候恢复数据库在事务开始前的状态，防止出现数据丢失或数据不一致的现象。如： 

In [22]:
try:
    stu = Stu(6,"NONAME")
    session.add(stu)
    session.flush()
    session.commit()
except Exception as e:
    print (type(e), e)
finally:
    session.rollback()
print (session.query(Stu).filter_by(id=6).first())

6: NONAME
